# GridWorld Value Function (4×4)

This notebook computes the state-value function \(V(s)\) for a 4×4 GridWorld under a uniform random policy (each action chosen with probability 0.25).  
Rewards are -1 per action step, the terminal state (bottom-right) has value 0, \(\gamma = 1\), and iteration stops when the maximum change in \(V\) falls below \(1e{-4}\).


## Setup

Imports and configuration values used throughout:
- Grid size \(N\), terminal state index
- Discount factor \(\gamma\)
- Convergence threshold \(\theta\)
- Action set (Up/Down/Left/Right) with equal probability
- NumPy print formatting for the final matrix


In [1]:
import numpy as np

# ----------------------------
# Configuration
# ----------------------------
N = 4                      # grid size N x N
GAMMA = 1.0                # no discounting
THETA = 1e-4               # convergence threshold
TERMINAL_STATE = N * N - 1 # bottom-right

# Actions: up, down, left, right (row_delta, col_delta)
ACTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]
ACTION_PROB = 1.0 / len(ACTIONS)  # equal probability for each move

# Print like the expected output
np.set_printoptions(precision=8, suppress=False)


## Transition dynamics

Defines the GridWorld mechanics:
- Converts between a 1D state index and (row, col)
- Applies an action with boundary handling (out-of-grid moves keep the agent in the same state)
- Uses reward -1 for every action attempt
- Treats the terminal state as absorbing with reward 0


In [2]:
def state_to_rc(s: int, n: int) -> tuple[int, int]:
    """Convert state index to (row, col)."""
    return divmod(s, n)

def rc_to_state(r: int, c: int, n: int) -> int:
    """Convert (row, col) back to state index."""
    return r * n + c

def step(s: int, action: tuple[int, int], n: int) -> tuple[int, float]:
    """
    Deterministic transition:
    - Terminal is absorbing with reward 0.
    - Every action attempt yields reward -1.
    - If action hits a boundary, agent stays in same state.
    """
    if s == TERMINAL_STATE:
        return s, 0.0

    r, c = state_to_rc(s, n)
    dr, dc = action
    nr, nc = r + dr, c + dc

    # Boundary handling: stay in place if out-of-bounds
    if nr < 0 or nr >= n or nc < 0 or nc >= n:
        ns = s
    else:
        ns = rc_to_state(nr, nc, n)

    return ns, -1.0


## Iterative Bellman updates

Runs iterative policy evaluation:
1. Initialize \(V(s)=0\) for all states.
2. For each non-terminal state, update using the Bellman expectation equation under a uniform random policy:  
   \(V_{\text{new}}(s) = \frac{1}{4}\sum_{a}\left[r(s,a) + \gamma V(s')\right]\)
3. Track the maximum absolute change across states.
4. Stop when the maximum change is below \(\theta = 1e{-4}\).


In [3]:
# Initialize V(s) = 0 for all states
V = np.zeros(N * N, dtype=np.float64)

iters = 0
while True:
    iters += 1
    delta = 0.0
    V_new = V.copy()

    for s in range(N * N):
        # Keep terminal state fixed at 0
        if s == TERMINAL_STATE:
            V_new[s] = 0.0
            continue

        # Bellman expectation update for uniform random policy
        v_s = 0.0
        for a in ACTIONS:
            ns, r = step(s, a, N)
            v_s += ACTION_PROB * (r + GAMMA * V[ns])

        V_new[s] = v_s
        delta = max(delta, abs(V_new[s] - V[s]))

    V = V_new

    # Convergence check
    if delta < THETA:
        break


## Final output

Reshapes the learned values into a 4×4 grid and prints:
- Iteration count
- Final max change (delta)
- The value function matrix (terminal cell should be 0)


In [4]:
V_grid = V.reshape(N, N)

print("Iterations:", iters)
print("Last max delta:", delta)
print(V_grid)


Iterations: 471
Last max delta: 9.891255676564015e-05
[[-59.42367735 -57.42387125 -54.2813141  -51.71012579]
 [-57.42387125 -54.56699476 -49.71029394 -45.13926711]
 [-54.2813141  -49.71029394 -40.85391609 -29.99766609]
 [-51.71012579 -45.13926711 -29.99766609   0.        ]]
